In [1]:
import pandas as pd
from functools import reduce

In [2]:
def clean_columns(df):
    df.columns = [col.replace("header_", "") if "header_" in col else col for col in df.columns ]
    if "matches" in list(df.columns):
        del df["matches"]
    return df

In [3]:
big5_std = clean_columns(pd.read_csv("./data/big5_standard_stats.csv"))
big5_def = clean_columns(pd.read_csv("./data/big5_defense_stats.csv"))
big5_att = clean_columns(pd.read_csv("./data/big5_gca_stats.csv"))
big5_misc = clean_columns(pd.read_csv("./data/big5_misc_stats.csv"))
big5_pass = clean_columns(pd.read_csv("./data/big5_passing_stats.csv"))
big5_play = clean_columns(pd.read_csv("./data/big5_playing_time.csv"))
big5_poss = clean_columns(pd.read_csv("./data/big5_possesion_stats.csv"))

In [4]:
stats_df = [big5_std, big5_def, big5_att, big5_misc, big5_pass, big5_play, big5_poss]

In [5]:
big5_att.columns

Index(['player', 'nationality', 'position', 'squad', 'comp_level', 'age',
       'birth_year', 'minutes_90s', 'sca_sca', 'sca_sca_per90',
       'sca_types_sca_passes_live', 'sca_types_sca_passes_dead',
       'sca_types_sca_dribbles', 'sca_types_sca_shots', 'sca_types_sca_fouled',
       'gca_gca', 'gca_gca_per90', 'gca_types_gca_passes_live',
       'gca_types_gca_passes_dead', 'gca_types_gca_dribbles',
       'gca_types_gca_shots', 'gca_types_gca_fouled', 'gca_types_gca_og_for'],
      dtype='object')

In [6]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['player', 'nationality', 'position', 'squad', 'comp_level', 'age', 'birth_year'],
                                            how='inner'), stats_df).fillna(-1)

In [7]:
df_merged.shape

(2712, 152)

In [8]:
drop_cols = list(set([col for col in df_merged.columns if (col.endswith("_y"))]))

In [9]:
rename_cols = list(set([col for col in df_merged.columns if (col.endswith("_x"))]))

In [10]:
def modifyCols(df, drop_cols, rename_cols):
    df = df.drop(drop_cols, axis=1)
    rename = [col.split("_x")[0] for col in rename_cols]
    rename = dict(zip(rename_cols, rename))
    df = df.rename(columns = rename)
    return df

In [11]:
df_modify = modifyCols(df_merged, drop_cols, rename_cols)

In [12]:
df_modify.shape

(2712, 145)

### Rename Positions

In [13]:
df_modify.position.unique()

array(['DF', 'FW,MF', 'MF', 'FW', 'DF,MF', 'GK', 'MF,FW', 'FW,DF',
       'MF,DF', 'DF,FW'], dtype=object)

In [14]:
df_modify.position = df_modify.position.replace({"FW,MF": "MF,FW", "FW,DF":"DF,FW", "MF,DF":"DF,MF"})

In [15]:
df_modify.position.unique()

array(['DF', 'MF,FW', 'MF', 'FW', 'DF,MF', 'GK', 'DF,FW'], dtype=object)

### Clean up Nationality

In [16]:
df_modify.nationality = df_modify.nationality.apply(lambda x: x.split(" ")[-1])

### Clean up comp_level

In [17]:
df_modify.comp_level = df_modify.comp_level.apply(lambda x: ' '.join(x.split(" ")[1:]))

In [18]:
import numpy as np

In [19]:
np.dtype('O')

dtype('O')

In [20]:
df_modify.columns[df_modify.dtypes.apply(lambda x: x == np.dtype('O'))]

Index(['player', 'nationality', 'position', 'squad', 'comp_level',
       'playing_minutes', 'per90_matches', 'xg_team_success_matches'],
      dtype='object')

### Modify data types of rest of the column to a floating point

In [21]:
non_numeric_cols = ['player', 'position', 'nationality', 'squad', 'comp_level']

for col in df_modify.columns:
    if col not in non_numeric_cols:
        df_modify[col] = df_modify[col].apply(lambda x: float(x.replace(',', '')) if type(x)=="str" else x)

In [25]:
list(df_modify.columns)

['player',
 'nationality',
 'position',
 'squad',
 'comp_level',
 'age',
 'birth_year',
 'games',
 'playing_games_starts',
 'playing_minutes',
 'playing_goals',
 'performance_assists',
 'performance_pens_made',
 'performance_pens_att',
 'performance_cards_yellow',
 'performance_cards_red',
 'performance_goals_per90',
 'per90_assists_per90',
 'per90_goals_assists_per90',
 'per90_goals_pens_per90',
 'per90_goals_assists_pens_per90',
 'per90_xg',
 'expected_npxg',
 'expected_xa',
 'expected_xg_per90',
 'per90_xa_per90',
 'per90_xg_xa_per90',
 'per90_npxg_per90',
 'per90_npxg_xa_per90',
 'per90_matches',
 'minutes_90s',
 'tackles_tackles',
 'tackles_tackles_won',
 'tackles_tackles_def_3rd',
 'tackles_tackles_mid_3rd',
 'tackles_tackles_att_3rd',
 'dribbles_def_dribble_tackles',
 'dribbles_def_dribbles_vs',
 'dribbles_def_dribble_tackles_pct',
 'dribbles_def_dribbled_past',
 'pressures_pressures',
 'pressures_pressure_regains',
 'pressures_pressure_regain_pct',
 'pressures_pressures_def_3rd

In [28]:
minutes = df_modify[["minutes_90s"]].iloc[:,1]
del df_modify["minutes_90s"]
df_modify["minutes_90"] = minutes

### Add Metadata about each column

In [29]:
df_modify.to_csv("./data/big5_full_stats.csv", index=False)